In [1]:
import os

In [2]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/tusharbhardwaj9873010398/Data-Science-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "tusharbhardwaj9873010398"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "08bbad566a9e25e40f9109c5d90704bbc8974a56"

In [3]:
%pwd

'c:\\Users\\tusha\\OneDrive\\Desktop\\Data-Science-Project\\research'

In [4]:
os.chdir('../')
%pwd

'c:\\Users\\tusha\\OneDrive\\Desktop\\Data-Science-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [6]:
from src.datascience.constant import *
from src.datascience.utils.comman import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: str = CONFIG_FILE_PATH,
                 params_filepath: str = PARAMS_FILE_PATH,
                 schema_filepath: str = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config["artifacts_root"]])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config["model_evaluation"]
        params = self.params["ElasticNet"]
        schema = self.schema["TARGET_COLUMN"]

        create_directories([config["root_dir"]])

        return ModelEvaluationConfig(
            root_dir=Path(config["root_dir"]),
            test_data_path=Path(config["test_data_path"]),
            model_path=Path(config["model_path"]),
            all_params=params,
            metric_file_name=Path(config["metrics_file"]),
            target_column=schema["name"],
            mlflow_uri=config.get("mlflow_uri", "default_mlflow_uri")
        )



In [8]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [9]:
import numpy as np
import pandas as pd
import joblib
import mlflow
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pathlib import Path

class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def evaluate_model(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop(columns=[self.config.target_column])
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            rmse, mae, r2 = self.evaluate_model(test_y, predicted_qualities)
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="model_v1")
            else:
                mlflow.sklearn.log_model(model, "model")


In [10]:
try:
    config_manager = ConfigurationManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-02-14 16:37:48,447: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2025-02-14 16:37:48,450: INFO: comman: yaml file: params.yaml loaded successfully]
[2025-02-14 16:37:48,454: INFO: comman: yaml file: schema.yaml loaded successfully]
[2025-02-14 16:37:48,456: INFO: comman: created directory at: artifacts]
[2025-02-14 16:37:48,457: INFO: comman: created directory at: artifacts/model_evalution]
[2025-02-14 16:37:49,380: INFO: comman: json file saved at: artifacts\model_evalution\metrics.json]


2025/02/14 16:37:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'model_v1' already exists. Creating a new version of this model...
Created version '2' of model 'model_v1'.


🏃 View run blushing-cat-744 at: https://dagshub.com/tusharbhardwaj9873010398/Data-Science-Project.mlflow/#/experiments/0/runs/1882f42733d345708039802a67178195
🧪 View experiment at: https://dagshub.com/tusharbhardwaj9873010398/Data-Science-Project.mlflow/#/experiments/0
